# 크롤링

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import requests
from urllib import request
import pandas as pd
from selenium.webdriver.common.by import By
import time
import pymysql
import hashlib 
import base64
head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

In [5]:
driver = webdriver.Chrome()
driver.get("https://www.inflearn.com/courses/it-programming?order=seq&page=1")
bs = BeautifulSoup(driver.page_source)
menu_list=[]# 각 메뉴 별 링크
menu = bs.findAll("div", class_='accordion-body')
for x in menu[:4]:
    menu = x.select_one('a')['href'] # 각 메뉴 별 href
    menu_list.append('https://www.inflearn.com' + menu )
menu_list

['https://www.inflearn.com/courses/it-programming?order=seq',
 'https://www.inflearn.com/courses/it?order=seq',
 'https://www.inflearn.com/courses/data-science?order=seq',
 'https://www.inflearn.com/courses/game-dev-all?order=seq']

### 1. 1순위, 리뷰 데이터

In [6]:
def text(x):
    return x.text

def main_url_html(f, main_url_page, soup_main_list): 
    '''f : 메뉴선택 0~3'''
    for i in range(1,page_range):
        main_urls = menu_list[f] + f'&page={i}' # 메뉴별 각 페이지 링크
        main_url_page.append(main_urls)
        main = requests.get(main_urls, headers=head)
        soup_main = BeautifulSoup(main.text,'html.parser')
        soup_main_list.append(soup_main) # 메인페이지 html
        
def course_url_fun(main_url_page, course_url):
    for main_url in main_url_page: 
        main = requests.get(main_url, headers=head)
        soup_main = BeautifulSoup(main.text,'html.parser')
        soup_main = soup_main.findAll("a", class_='course_card_front e_course_click') # 각 강의들
        for y in soup_main:
            course_url.append('https://www.inflearn.com' + y['href']) # 강의별 링크
# uuid            
def uuid_fun(course_url, uuid_list):
    for url_s in course_url:
        uuid = hashlib.sha256(url_s.encode()) 
        uuid_list.append(uuid.hexdigest())
# 강의명
def course_fun(course_url, title_list):
    for course_ur in course_url:
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        replace_word = soup_course.select_one(selector="h1.cd-header__title span").text
        course_name = soup_course.select_one(selector="h1.cd-header__title").text.replace(f' {replace_word}','')
        course_name = re.sub('[^A-Za-z0-9가-힣+]', ' ',course_name)
        title_list.append(course_name)
        
# 카테고리
def categorie_fun(course_url, categorie_list):
    for course_ur in course_url:
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        categories = soup_course.select(selector="span.cd-header__breadcrumb-el")
        categorie_list.append(list(map(text,categories)))
        categorie_list.append([re.sub('[^A-Za-z0-9가-힣+]', ' ',i.strip()) for i in categories_map])
        
# 핵심 키워드
def keyword_fun(course_url, keyword_list):
    for course_ur in course_url:
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        keyword = soup_course.select(selector="a.cd-header__tag")
        keyword_map = list(map(text,keyword))
        keyword_list.append([re.sub('[^A-Za-z0-9가-힣+]', ' ',i.strip()) for i in keyword_map])
        
# 강의 소개
def course_info_fun(course_url, course_info_list):
    for course_ur in course_url:
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        course = soup_course.select(selector="section.cd-body")[0].text
        course_info_list.append(re.sub('[^A-Za-z0-9가-힣+]', ' ', course))
        
# url-ID
def course_id(soup_main_list, course_id_list):
    for soup_main_html in soup_main_list:
        course_ids = soup_main_html.select("div.card")
        for course_id in course_ids:
            course_id_list.append(course_id["data-productid"])
# 리뷰
def review(course_id_list, review_list):
    for course_id in course_id_list:
        url = f"https://www.inflearn.com/api/v1/course/{course_id}/review?pageSize=10000&pageNumber=1"
        rq = requests.get(url, headers = head)
        review_data = json.loads(rq.text)
        rev_list = [re.sub('[^A-Za-z0-9가-힣+]', ' ',reviews['body']) for reviews in review_data['data']['items']]
        review_list.append(", ".join(rev_list).replace(',',' '))

In [7]:
# 개발,프로그래밍
page_range = 54  # 강의 페이지 정하기 최대 53까지 있음
main_url_page0 = [] # 페이지별 링크
soup_main_list0 = [] # 페이지별 html
course_url0 = [] # 강의별 링크

uuid_list0 = []
title_list0 = []
categorie_list0 = []
keyword_list0 = []
course_info_list0 = []
course_id_list0 = []
review_list0 = []

main_url_html(0, main_url_page0, soup_main_list0)  
course_url_fun(main_url_page0, course_url0)

uuid_fun(course_url0, uuid_list0)
course_fun(course_url0, title_list0)
categorie_fun(course_url0, categorie_list0)
keyword_fun(course_url0, keyword_list0)
course_info_fun(course_url0, course_info_list0)

course_id(soup_main_list0, course_id_list0)
review(course_id_list0, review_list0)

In [ ]:
# 리스트 풀기
categorie_list00 = []
for i in categorie_list0:
    categorie_list00.append(" / ".join(i))
    
keyword_list00 = []
for i in keyword_list0:
    keyword_list00.append(" / ".join(i))

In [ ]:
# 보안, 네트워크
page_range = 10  # 강의 페이지 정하기 최대 9까지 있음
main_url_page1 = [] # 페이지별 링크
soup_main_list1 = [] # 페이지별 html
course_url1 = [] # 강의별 링크

uuid_list1 = []
title_list1 = []
categorie_list1 = []
keyword_list1 = []
course_info_list1 = []
course_id_list1 = []
review_list1 = []
 
main_url_html(1, main_url_page1, soup_main_list1)
course_url_fun(main_url_page1, course_url1)

uuid_fun(course_url1, uuid_list1)
course_fun(course_url1, title_list1)
categorie_fun(course_url1, categorie_list1)
keyword_fun(course_url1, keyword_list1)
course_info_fun(course_url1, course_info_list1)

course_id(soup_main_list1, course_id_list1)
review(course_id_list1, review_list1)

In [ ]:
categorie_list11 = []
for i in categorie_list1:
    categorie_list11.append(" / ".join(i))
    
keyword_list11 = []
for i in keyword_list1:
    keyword_list11.append(" / ".join(i))

In [ ]:
# 데이터 사이언스
page_range = 15  # 강의 페이지 정하기 최대 14까지 있음
main_url_page2 = [] # 페이지별 링크
soup_main_list2 = [] # 페이지별 html
course_url2 = [] # 강의별 링크

uuid_list2 = []
title_list2 = []
categorie_list2 = []
keyword_list2 = []
course_info_list2 = []
course_id_list2 = []
review_list2 = []

main_url_html(2, main_url_page2, soup_main_list2)
course_url_fun(main_url_page2, course_url2)

uuid_fun(course_url2, uuid_list2)
course_fun(course_url2, title_list2)
categorie_fun(course_url2, categorie_list2)
keyword_fun(course_url2, keyword_list2)
course_info_fun(course_url2, course_info_list2)

course_id(soup_main_list2, course_id_list2)
review(course_id_list2, review_list2)

In [ ]:
categorie_list22 = []
for i in categorie_list2:
    categorie_list22.append(" / ".join(i))
    
keyword_list22 = []
for i in keyword_list2:
    keyword_list22.append(" / ".join(i))

In [ ]:
# 게임 개발
page_range = 7  # 강의 페이지 정하기 최대 6까지 있음
main_url_page3 = [] # 페이지별 링크
soup_main_list3 = [] # 페이지별 html
course_url3 = [] # 강의별 링크

uuid_list3 = []
title_list3 = []
categorie_list3 = []
keyword_list3 = []
course_info_list3 = []
course_id_list3 = []
review_list3 = []

main_url_html(3, main_url_page3, soup_main_list3)
course_url_fun(main_url_page3, course_url3)

uuid_fun(course_url3, uuid_list3)
course_fun(course_url3, title_list3)
categorie_fun(course_url3, categorie_list3)
keyword_fun(course_url3, keyword_list3)
course_info_fun(course_url3, course_info_list3)

course_id(soup_main_list3, course_id_list3)
review(course_id_list3, review_list3)

In [ ]:
categorie_list33 = []
for i in categorie_list3:
    categorie_list33.append(" / ".join(i))
    
keyword_list33 = []
for i in keyword_list3:
    keyword_list33.append(" / ".join(i))

In [ ]:
# html base64로 암호화
soup_main_list00 = []

for soup_main in soup_main_list0:
    soup_main_list00.append([base64.b64encode(str(soup_main).encode())])

In [ ]:
soup_main_list11 = []

for soup_main in soup_main_list1:
    soup_main_list11.append([base64.b64encode(str(soup_main).encode())])

In [ ]:
soup_main_list22 = []

for soup_main in soup_main_list2:
    soup_main_list22.append([base64.b64encode(str(soup_main).encode())])

In [ ]:
soup_main_list33 = []

for soup_main in soup_main_list3:
    soup_main_list33.append([base64.b64encode(str(soup_main).encode())])

### 2. 2순위 데이터

In [ ]:
# 평점, 수강생수, 강사명, 소요시간, 난이도, 가격

def grade(course_url, grade_list):
    for course_ur in course_url:
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        if soup_course.select(selector="span.cd-header__info--star strong"):
            grade = soup_course.select(selector="span.cd-header__info--star strong")[0].text
            grade_list.append(grade.replace('(','').replace(')',''))
        else:
            grade_list.append(0)
        
def stu_num(course_url, stu_numl_list):
    for course_ur in course_url:
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        if soup_course.select(selector="div.cd-header__info-cover"):
            stu_num = soup_course.select(selector="div.cd-header__info-cover strong")[-1].text.replace('명','')
            stu_numl_list.append(stu_num)
        else:
            stu_num = soup_course.select(selector="div.cd-header__student_cnt strong")[0].text.replace('명','')
            stu_numl_list.append(stu_num)
        
def teacher(course_url, teacher_list):
    for course_ur in course_url:
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        teacher = soup_course.select(selector="a.cd-header__instructors--main")[0].text.strip()
        teacher_list.append(re.sub('[^A-Za-z0-9가-힣+]', ' ',teacher))
        
def playtime(course_url, time_list):
    for course_ur in course_url:
        try:
            soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
            time = soup_course.select(selector="div.cd-floating__info-row")[1].text.split('(')[1].split(')')[0]
            time_list.append(float(time.replace('시간 ','.').replace('분','')))
        except:
            time_list.append(0)
        
def lebel(course_url, lebel_list):
    for course_ur in course_url:
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        for i_gray in soup_course.select(selector="div.cd-floating__info-row")[4].select('div.cd-floating__info--gray'):
            i_gray.decompose()
        if soup_course.select(selector="div.cd-floating__info-row")[4].select("div.cd-floating__info-tooltip"):
            for i_lebel in soup_course.select(selector="div.cd-floating__info-row")[4].select("div.cd-floating__info-tooltip"):
                if "입문" in i_lebel.text:
                    lebel_list.append(1)
                elif "초급" in i_lebel.text:
                    lebel_list.append(2)
                else:
                    lebel_list.append(3)
        else:
            lebel_list.append(0)
        
def price(course_url, price_list):
    for course_ur in course_url:
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        if "무료" in soup_course.select(selector="div.cd-floating__card--top h4")[0].text: # 무료
            price_list.append(0)
        elif soup_course.select(selector="div.cd-floating__price--top"): # 할부
            price = soup_course.select(selector="div.cd-floating__price--top")[0].select("span.cd-price__pay-price")[0].text.replace("원",'').replace(',','')
            price = int(price)
            price_list.append(price)
        elif soup_course.select(selector="h4.cd-price__discount-rate"): # 할인
            price = soup_course.select(selector="div.cd-floating__card--top h4")[1].text.replace("원",'').replace(',','')
            price = int(price)
            price_list.append(price)
        else:
            price = soup_course.select(selector="div.cd-floating__card--top h4")[0].text.replace("원",'').replace(',','')
            price = int(price)
            price_list.append(price)

In [ ]:
grade_list0 = []
stu_numl_list0 = []
teacher_list0 = []
time_list0 = []
lebel_list0 = []
price_list0 = []

grade(course_url0, grade_list0)
stu_num(course_url0, stu_numl_list0)
teacher(course_url0, teacher_list0)
playtime(course_url0, time_list0)
lebel(course_url0, lebel_list0)
price(course_url0, price_list0)

In [ ]:
grade_list1 = []
stu_numl_list1 = []
teacher_list1 = []
time_list1 = []
lebel_list1 = []
price_list1 = []

grade(course_url1, grade_list1)
stu_num(course_url1, stu_numl_list1)
teacher(course_url1, teacher_list1)
time(course_url1, time_list1)
lebel(course_url1, lebel_list1)
price(course_url1, price_list1)

In [ ]:
grade_list2 = []
stu_numl_list2 = []
teacher_list2 = []
time_list2 = []
lebel_list2 = []
price_list2 = []

grade(course_url2, grade_list2)
stu_num(course_url2, stu_numl_list2)
teacher(course_url2, teacher_list2)
time(course_url2, time_list2)
lebel(course_url2, lebel_list2)
price(course_url2, price_list2)

In [ ]:
grade_list3 = []
stu_numl_list3 = []
teacher_list3 = []
time_list3 = []
lebel_list3 = []
price_list3 = []

grade(course_url3, grade_list3)
stu_num(course_url3, stu_numl_list3)
teacher(course_url3, teacher_list3)
time(course_url3, time_list3)
lebel(course_url3, lebel_list3)
price(course_url3, price_list3)

In [ ]:
pair_list0 = []
[pair_list0.append(list(pair)) for pair in zip(uuid_list0,course_url0, title_list0, categorie_list00, keyword_list00, course_info_list0, original_data00, course_id_list0, review_list0)]
pair_list1 = []
[pair_list1.append(list(pair)) for pair in zip(uuid_list1,course_url1, title_list1, categorie_list11, keyword_list11, course_info_list1, original_data11, course_id_list1, review_list1)]
pair_list2 = []
[pair_list2.append(list(pair)) for pair in zip(uuid_list2,course_url2, title_list2, categorie_list22, keyword_list22, course_info_list2, original_data22, course_id_list2, review_list2)]
pair_list3 = []
[pair_list3.append(list(pair)) for pair in zip(uuid_list3,course_url3, title_list3, categorie_list33, keyword_list33, course_info_list3, original_data33, course_id_list3, review_list3)]

In [ ]:
pair_list = pair_list0+pair_list1+pair_list2+pair_list3

In [ ]:
pair_list_columns = ['uuid', 'url', 'course_data', 'categorie_data', 'keyword_data', 'course_info_data', 'ori_data', 'course_id', 'review']

df_pair_list= pd.DataFrame(pair_list, columns = pair_list_columns)

# DB

In [1]:
# 테이블 생성
import pymysql
# not null primary key,
db = pymysql.connect(host='localhost', port=3306, user='root', passwd=passwd, db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
    create table chat_bot_001 (
        uuid varchar(100) not null primary key, 
        url varchar(100),
        course_data varchar(300),
        categorie_data varchar(300),
        keyword_data varchar(300),
        course_info_data text,
        course_id int(10),
        review_data longtext,
        grade float(24),
        stu_num int(10),
        teacher varchar(50),
        playtime_data float(24),
        level_data int(10),
        price_data int(10),
        course varchar(300) ,
        categorie varchar(300),
        keyword varchar(300),
        course_info text,
        review longtext,
        dibs_num int(10),
        teacher_info text,
        teacher_info_data text        
    );
"""
cursor.execute(sql)
db.commit()
db.close()

In [ ]:
# 중복있는 df : chat_bot_001
db = pymysql.connect(host='localhost', port=3306, user='root', passwd=passwd, db='mywork', charset='utf8')
cursor = db.cursor()
#14개 컬럼
sql = """INSERT INTO chat_bot_001 (uuid, url, course_data, categorie_data,keyword_data,course_info_data,course_id,review_data,grade,stu_num,teacher,playtime_data,level_data, price_data) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"""

for i, row in df_pair_list.iterrows():
    cursor.execute(sql, tuple(row))
    db.commit()

# 형태소 분석기

In [ ]:
komoran = Komoran(userdic='./total_set.csv')

In [ ]:
df2 = df_pair_list.copy()

In [ ]:
df2['uuid'].nunique()
df3 = df2.drop_duplicates(subset = ["uuid"])

In [ ]:
word_list = []
with open('total_set.csv', 'r', encoding='UTF-8') as f:
        lines = f.readlines()
        for line in lines:
                 word_list.append(line.rstrip())
word_list_lower = [i.lower() for i in word_list]

In [ ]:
# 소문자로 변환 후 비교
data_course_lower = []
for morphs in df3['course_data'].apply(komoran.morphs):
    using_word = []
    using_word_join = []
    for word in morphs:
        if word.lower() in word_list_lower:
            using_word.append(word)
    using_word_join.append(', '.join(using_word))
    data_course_lower.append(using_word_join)

data_review_lower = []
for morphs in df3['review_data'].apply(komoran.morphs):
    using_word = []
    using_word_join = []
    for word in morphs:
        if word.lower() in word_list_lower:
            using_word.append(word)
    using_word_join.append(', '.join(using_word))
    data_review_lower.append(using_word_join)

data_categorie_lower = []
for morphs in df3['categorie_data'].apply(komoran.morphs):
    using_word = []
    using_word_join = []
    for word in morphs:
        if word.lower() in word_list_lower:
            using_word.append(word)
    using_word_join.append(', '.join(using_word))
    data_categorie_lower.append(using_word_join)
    
data_keyword_lower = []
for morphs in df3['keyword_data'].apply(komoran.morphs):
    using_word = []
    using_word_join = []
    for word in morphs:
        if word.lower() in word_list_lower:
            using_word.append(word)
    using_word_join.append(', '.join(using_word))
    data_keyword_lower.append(using_word_join)

data_info_lower = []
for morphs in df3['course_info_data'].apply(komoran.morphs):
    using_word = []
    using_word_join = []
    for word in morphs:
        if word.lower() in word_list_lower:
            using_word.append(word)
    using_word_join.append(', '.join(using_word))
    data_info_lower.append(using_word_join)

In [ ]:
data_course_df= pd.DataFrame(data_course_lower, columns =['data_course_df'])
data_review_df= pd.DataFrame(data_review_lower, columns =['data_review_df'])
data_categorie_df= pd.DataFrame(data_categorie_lower, columns =['data_categorie_df'])
data_keyword_df= pd.DataFrame(data_keyword_lower, columns =['data_keyword_df'])
data_info_df= pd.DataFrame(data_info_lower, columns =['data_info_df'])

In [ ]:
concat_pd = pd.concat([data_course_df,data_review_df],axis=1)

In [ ]:
concat_pd1 = concat_pd.copy()

In [ ]:
concat_pd1['data_categorie_df'] =data_categorie_df
concat_pd1['data_keyword_df']=data_keyword_df
concat_pd1['data_info_df']=data_info_df

In [ ]:
df3.reset_index(inplace=True)

In [ ]:
concat_1581=pd.concat([df3,concat_pd1],axis=1)
concat_1581 = concat_1581.replace({np.nan: 0})
concat_1581= concat_1581.drop(['index'],axis=1)

In [ ]:
# dataframe > sql삽입
db = pymysql.connect(host='localhost', port=3306, user='root', passwd=passwd, db='mywork', charset='utf8')
cursor = db.cursor()
# 19개 컬럼
sql = """INSERT INTO chat_bot_002 (uuid, url, course_data, categorie_data,keyword_data,course_info_data,course_id,review_data,grade,stu_num,teacher,playtime_data,level_data, price_data,course,review,categorie,keyword,course_info) 
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"""

for i, row in concat_1581.iterrows():
    cursor.execute(sql, tuple(row))
  #  print(tuple(row))
    db.commit()

# 3순위 데이터 작업

### DB에서 html꺼내기

In [2]:
# main url 4개의 각 페이지별 html을 파싱 후 따로 db테이블에 저장해 둠
# 조회 - main html
main_html = []

db = pymysql.connect(host='localhost', port=3306, user='root', passwd=passwd, db='mywork', charset='utf8')
cursor = db.cursor()

sql = "SELECT main_html FROM chat_bot_main_html"
cursor.execute(sql)
result = cursor.fetchall()
for datas in result:
    main_html.append(datas)
    
cursor.close()
db.close()

In [3]:
main_ht = []
for html_tuple in main_html:
    for i in html_tuple:
        str_bytes= base64.b64decode(i)
        init_str = str_bytes.decode('utf-8')
        main_ht.append(init_str)

In [4]:
len(main_ht)

82

In [5]:
# 메인 html (html = soup_main_list )
def main_url_html(main_ht, soup_main_list): 
    nums = 0
    for i in main_ht:
        nums  += 1
        if nums%5 ==0:
            print(nums)
        soup_main = BeautifulSoup(i,'html.parser')
        soup_main_list.append(soup_main) # 메인페이지 html

In [8]:
soup_main_list = []
main_url_html(main_ht, soup_main_list)

len(soup_main_list)

82

In [ ]:
def text(x):
    return x.text

# 강의별 링크
def course_url_fun(soup_main_list, course_url):
    nums = 0
    for soup_main_html in soup_main_list: 
        nums  += 1
        if nums%2 ==0:
            print(nums)
        soup_main = soup_main_html.findAll("a", class_='course_card_front e_course_click') # 각 강의들
        for y in soup_main:
            course_url.append('https://www.inflearn.com' + y['href'])
# uuid            
def uuid_fun(course_url, uuid_list):
    nums = 0
    for url_s in course_url:
        nums  += 1
        if nums%5 ==0:
            print(nums)
        uuid = hashlib.sha256(url_s.encode()) 
        uuid_list.append(uuid.hexdigest())

In [12]:
course_url = [] 
course_url_fun(soup_main_list, course_url)

len(course_url)

1931

In [ ]:
uuid_list = []
uuid_fun(course_url, uuid_list)

In [28]:
# 찜 개수
def dibs_num(course_url, dibs_num_list):
    nums = 0
    for course_ur in course_url:
        nums  += 1
        if nums%5 ==0:
            print(nums)
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        if soup_course.select_one(selector="span.cd-floating__sub-button--cnt"):
            dibs_num = soup_course.select_one(selector="span.cd-floating__sub-button--cnt").text.replace('K','000').replace('.','')
            dibs_num_list.append(dibs_num)
        else:
            dibs_num_list.append(0)
            
# 강사(선생님) 소개            
def teacher_info(course_url, teacher_info_list):
    nums = 0
    for course_ur in course_url:
        nums  += 1
        if nums%5 ==0:
            print(nums)
        soup_course = BeautifulSoup(requests.get(course_ur).text,'html.parser')
        teacher_info = soup_course.select_one(selector="div.cd-instructors__cover").text.replace('더 읽기','').strip()
        teacher_info_list.append(teacher_info)

In [30]:
dibs_num_list = []
dibs_num(course_url, dibs_num_list)

len(dibs_num_list)

1931

In [43]:
teacher_info_list = []
teacher_info(course_url, teacher_info_list)

len(teacher_info_list)

1931

In [103]:
# 테이블 생성
db = pymysql.connect(host='localhost', port=3306, user='root', passwd=passwd, db='mywork', charset='utf8')
cursor = db.cursor()
sql = """
    create table chat_bot_004 (
        uuid varchar(100) not null primary key, 
        url varchar(100),
        course_data varchar(300),
        categorie_data varchar(300),
        keyword_data varchar(300),
        course_info_data text,
        course_id int(10),
        review_data longtext,
        grade float(24),
        stu_num int(10),
        teacher varchar(50),
        playtime_data float(24),
        level_data int(10),
        price_data int(10),
        course varchar(300) ,
        categorie varchar(300),
        keyword varchar(300),
        course_info text,
        review longtext,
        dibs_num int(10),
        teacher_info text,
        teacher_info_data text        
    );
"""
cursor.execute(sql)
db.commit()
db.close()

In [53]:
# 조회 - chat_bot_002
import pymysql
chat_bot_002 = []

db = pymysql.connect(host='localhost', port=3306, user='root', passwd=passwd, db='mywork', charset='utf8')
cursor = db.cursor()

sql = "SELECT * FROM chat_bot_002"
cursor.execute(sql)
result = cursor.fetchall()
for datas in result:
    chat_bot_002.append(datas)
    
cursor.close()
db.close()

In [56]:
df_002= pd.DataFrame(chat_bot_002)
df_002.drop([19,20],axis=1,inplace=True)

In [65]:
pair_list_columns = ['uuid', 'url', 'course_data', 'categorie_data', 'keyword_data', 'course_info_data', 'course_id', 'review_data','grade','stu_num','teacher','time','level','price','course', 'categorie', 'keyword', 'course_info', 'review']
df_002.columns =pair_list_columns
df_002.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1581 entries, 0 to 1580
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uuid              1581 non-null   object 
 1   url               1581 non-null   object 
 2   course_data       1581 non-null   object 
 3   categorie_data    1581 non-null   object 
 4   keyword_data      1581 non-null   object 
 5   course_info_data  1581 non-null   object 
 6   course_id         1581 non-null   int64  
 7   review_data       1581 non-null   object 
 8   grade             1581 non-null   float64
 9   stu_num           1581 non-null   int64  
 10  teacher           1581 non-null   object 
 11  time              1581 non-null   float64
 12  level             1581 non-null   int64  
 13  price             1581 non-null   int64  
 14  course            1581 non-null   object 
 15  categorie         1581 non-null   object 
 16  keyword           1581 non-null   object 


In [45]:
pair_list = [list(pair) for pair in zip(uuid_list, dibs_num_list, teacher_info_list)]

In [48]:
pair_list_columns = ['uuid', 'dibs_num', 'teacher_info_data']
df= pd.DataFrame(pair_list, columns = pair_list_columns)

In [51]:
df2 = df.drop_duplicates(subset = ["uuid"])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1581 entries, 0 to 1925
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   uuid               1581 non-null   object
 1   dibs_num           1581 non-null   object
 2   teacher_info_data  1581 non-null   object
dtypes: object(3)
memory usage: 49.4+ KB


In [67]:
df2.reset_index(inplace=True)
df_002.reset_index(inplace=True)

In [69]:
df3 = pd.merge(df_002, df2, on = 'uuid', how = 'outer')

In [74]:
df3.drop(['index_x'],axis=1,inplace=True)
df3.drop(['index_y'],axis=1,inplace=True)

In [78]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1581 entries, 0 to 1580
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   uuid               1581 non-null   object 
 1   url                1581 non-null   object 
 2   course_data        1581 non-null   object 
 3   categorie_data     1581 non-null   object 
 4   keyword_data       1581 non-null   object 
 5   course_info_data   1581 non-null   object 
 6   course_id          1581 non-null   int64  
 7   review_data        1581 non-null   object 
 8   grade              1581 non-null   float64
 9   stu_num            1581 non-null   int64  
 10  teacher            1581 non-null   object 
 11  time               1581 non-null   float64
 12  level              1581 non-null   int64  
 13  price              1581 non-null   int64  
 14  course             1581 non-null   object 
 15  categorie          1581 non-null   object 
 16  keyword            1581 

In [81]:
komoran = Komoran(userdic='./total_set.csv')

In [82]:
df4 = df3.copy()

In [83]:
word_list = []
with open('total_set.csv', 'r', encoding='UTF-8') as f:
        lines = f.readlines()
        for line in lines:
                 word_list.append(line.rstrip())

In [84]:
word_list_lower = [i.lower() for i in word_list]

In [87]:
df3['teacher_info_data'] = df3['teacher_info_data'].str.replace(pat=r'[^A-Za-z0-9가-힣+]', repl=r' ', regex=True) 

In [90]:
# 소문자로 변환 후 비교 word_list_lower
data_teacher_info_lower = []
for morphs in df3['teacher_info_data'].apply(komoran.morphs):
    using_word = []
    using_word_join = []
    for word in morphs:
        if word.lower() in word_list_lower:
            using_word.append(word)
    using_word_join.append(', '.join(using_word))
    data_teacher_info_lower.append(using_word_join)

In [95]:
len(data_teacher_info_lower)

1581

In [96]:
data_teacher_info_df= pd.DataFrame(data_teacher_info_lower, columns =['data_teacher_info_df'])

In [99]:
concat_pd = pd.concat([df3,data_teacher_info_df],axis=1)

In [100]:
concat_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1581 entries, 0 to 1580
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   uuid                  1581 non-null   object 
 1   url                   1581 non-null   object 
 2   course_data           1581 non-null   object 
 3   categorie_data        1581 non-null   object 
 4   keyword_data          1581 non-null   object 
 5   course_info_data      1581 non-null   object 
 6   course_id             1581 non-null   int64  
 7   review_data           1581 non-null   object 
 8   grade                 1581 non-null   float64
 9   stu_num               1581 non-null   int64  
 10  teacher               1581 non-null   object 
 11  time                  1581 non-null   float64
 12  level                 1581 non-null   int64  
 13  price                 1581 non-null   int64  
 14  course                1581 non-null   object 
 15  categorie            

In [104]:
# dataframe > sql삽입
db = pymysql.connect(host='localhost', port=3306, user='root', passwd=passwd, db='mywork', charset='utf8')
cursor = db.cursor()
#14개 컬럼
sql = """INSERT INTO chat_bot_004 
(uuid, url, course_data, categorie_data,keyword_data,course_info_data,course_id,review_data,grade,stu_num,teacher,playtime_data,
level_data, price_data,course,categorie,keyword,course_info,review,dibs_num,teacher_info,teacher_info_data) 
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"""

for i, row in concat_pd.iterrows():
    cursor.execute(sql, tuple(row))
    db.commit()